<a href="https://colab.research.google.com/github/iam-fern/Generate_data/blob/main/ex_sampling_c_1000rep_n30_x6_sas.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans

import statistics as stats
import scipy.stats as ss
import scipy.stats as st
from scipy.stats import f_oneway
from scipy import stats
import scipy.stats as stats
import time
import statsmodels.api as sm

from statsmodels.formula.api import ols

/usr/local/lib/python3.7/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [2]:
df = pd.read_csv('https://github.com/iam-fern/Research/raw/main/sampling_1000rep_n30_x6_sas.csv', index_col=['rep', 'index'])
df

x1         x2  ...         y_hat         e_hat
rep   index                        ...                            
0.0   0.0    49.064013  11.621527  ...  1.859669e+08 -2.233135e+10
      1.0    46.163827  48.471380  ...  4.707361e+07  2.507011e+10
      2.0    19.565028  45.358574  ... -6.947298e+07  8.201673e+07
      3.0    47.207795  33.529469  ...  1.501501e+08  1.482675e+10
      4.0    38.531559  20.957039  ...  1.517231e+08 -3.735350e+07
...                ...        ...  ...           ...           ...
999.0 25.0   38.998576  17.843815  ...  8.588873e+07  4.704133e+09
      26.0   20.418946  48.610952  ... -5.432296e+07  4.642706e+07
      27.0   48.550374  46.434238  ...  6.074838e+07 -4.209132e+09
      28.0   23.742248  15.709370  ...  9.533341e+06  3.579106e+07
      29.0   36.105864  43.607732  ... -1.149690e+07 -1.739795e+09

[30000 rows x 7 columns]

In [3]:
# ลบข้อมูลที่มี missing values
table = df.dropna()
table

x1         x2  ...         y_hat         e_hat
rep   index                        ...                            
0.0   0.0    49.064013  11.621527  ...  1.859669e+08 -2.233135e+10
      1.0    46.163827  48.471380  ...  4.707361e+07  2.507011e+10
      2.0    19.565028  45.358574  ... -6.947298e+07  8.201673e+07
      3.0    47.207795  33.529469  ...  1.501501e+08  1.482675e+10
      4.0    38.531559  20.957039  ...  1.517231e+08 -3.735350e+07
...                ...        ...  ...           ...           ...
999.0 25.0   38.998576  17.843815  ...  8.588873e+07  4.704133e+09
      26.0   20.418946  48.610952  ... -5.432296e+07  4.642706e+07
      27.0   48.550374  46.434238  ...  6.074838e+07 -4.209132e+09
      28.0   23.742248  15.709370  ...  9.533341e+06  3.579106e+07
      29.0   36.105864  43.607732  ... -1.149690e+07 -1.739795e+09

[30000 rows x 7 columns]

In [4]:
table.reset_index(inplace=True)
table

,rep,index,x1,x2,x3,e,Y,y_hat,e_hat
0,0.0,0.0,49.064013,11.621527,21.302205,-2.214538e+10,-2.214538e+10,1.859669e+08,-2.233135e+10
1,0.0,1.0,46.163827,48.471380,19.223355,2.511719e+10,2.511719e+10,4.707361e+07,2.507011e+10
2,0.0,2.0,19.565028,45.358574,26.960437,1.254348e+07,1.254375e+07,-6.947298e+07,8.201673e+07
3,0.0,3.0,47.207795,33.529469,36.965787,1.497690e+10,1.497691e+10,1.501501e+08,1.482675e+10
4,0.0,4.0,38.531559,20.957039,40.531290,1.143692e+08,1.143696e+08,1.517231e+08,-3.735350e+07
...,...,...,...,...,...,...,...,...,...
29995,999.0,25.0,38.998576,17.843815,12.665639,4.790021e+09,4.790022e+09,8.588873e+07,4.704133e+09
29996,999.0,26.0,20.418946,48.610952,34.329912,-7.896197e+06,-7.895905e+06,-5.432296e+07,4.642706e+07
29997,999.0,27.0,48.550374,46.434238,17.059402,-4.148384e+09,-4.148384e+09,6.074838e+07,-4.209132e+09
29998,999.0,28.0,23.742248,15.709370,13.326378,4.532413e+07,4.532440e+07,9.533341e+06,3.579106e+07


In [5]:
table.set_index(['rep', 'index'], inplace=True)

In [6]:
table

x1         x2  ...         y_hat         e_hat
rep   index                        ...                            
0.0   0.0    49.064013  11.621527  ...  1.859669e+08 -2.233135e+10
      1.0    46.163827  48.471380  ...  4.707361e+07  2.507011e+10
      2.0    19.565028  45.358574  ... -6.947298e+07  8.201673e+07
      3.0    47.207795  33.529469  ...  1.501501e+08  1.482675e+10
      4.0    38.531559  20.957039  ...  1.517231e+08 -3.735350e+07
...                ...        ...  ...           ...           ...
999.0 25.0   38.998576  17.843815  ...  8.588873e+07  4.704133e+09
      26.0   20.418946  48.610952  ... -5.432296e+07  4.642706e+07
      27.0   48.550374  46.434238  ...  6.074838e+07 -4.209132e+09
      28.0   23.742248  15.709370  ...  9.533341e+06  3.579106e+07
      29.0   36.105864  43.607732  ... -1.149690e+07 -1.739795e+09

[30000 rows x 7 columns]

In [7]:
def get_result(run_replicate):
    new_table = table.loc[run_replicate]

    # show index
    new_table.reset_index(inplace=True)

    new_table.drop(['index'], axis=1, inplace=True)


    # แบ่งข้อมูลออกเป็น 2 กลุ่มด้วย k-mean

    #Set number of clusters
    x = new_table[["x1", "x2", "x3"]]
    k_means = KMeans(n_clusters=2)

    #Run the clustering algorithm
    model = k_means.fit(x)
    model

    #Generate cluster predictions and store in y_kmeans
    y_kmeans = k_means.predict(x)
    y_kmeans

    # kmean_clusters.astype(float)
    kmean_clusters = pd.DataFrame({"clusters" : y_kmeans})

    # เชื่อมข้อมูลทั้ง 2 DataFrame เข้าด้วยกันด้วยคำสั่ง pd.concat
    data_clusters = pd.concat([new_table, kmean_clusters], axis=1)

    data_sort_clusters = data_clusters.sort_values('clusters')

    # print(data_clusters.groupby('clusters').groups)

    data_sort_clusters.reset_index(inplace=True)

    # กลุ่มที่ 1
    data_cluster1 = data_sort_clusters[data_sort_clusters.clusters == 1]

    y = data_cluster1[['Y']]
    x = data_cluster1[['x1', 'x2', 'x3']]
    x = sm.add_constant(x)
    model1 = sm.OLS(y, x).fit()

    DF1 = model1.df_resid

    SSR1 = model1.ssr

    MSE1 = model1.mse_resid

    # กลุ่มที่ 2
    data_cluster0 = data_sort_clusters[data_sort_clusters.clusters == 0]

    y = data_cluster0[['Y']]
    x = data_cluster0[['x1', 'x2', 'x3']]
    x = sm.add_constant(x)
    model0 = sm.OLS(y, x).fit()

    DF0 = model0.df_resid

    SSR0 = model0.ssr

    MSE0 = model0.mse_resid

    df1 = DF1+1
    df0 = DF0+1

    # คำนวณค่าสถิติ
    GQ_test_clusters = (SSR1/df1) / (SSR0/df0)


    # คำนวณค่า p-value
    # sf(x, dfn, dfd, loc=0, scale=1) : Survival function (also defined as 1 - cdf, but sf is sometimes more accurate)
    two_tailed = 2.0*(1.0-(ss.f.cdf(GQ_test_clusters, df1, df0)))

    # pdf(x, dfn, dfd, loc=0, scale=1) : Probability density function
    p_greater = ss.f.cdf(GQ_test_clusters, df1, df0)
    
    return df0, df1, SSR0, SSR1, MSE0, MSE1, GQ_test_clusters, two_tailed, p_greater

# สร้าง list ของ replicates
list_replicate=list({i[0] for i in table.index})
list_replicate.sort()

# สร้าง list เพื่อเก็บค่า ทุกรอบที่ run
list_df0=[]
list_df1=[]
list_SSR0=[]
list_SSR1=[]
list_MSE0=[]
list_MSE1=[]
list_GQ_test_clusters=[]
list_result_test=[]
list_two_tailed=[]
list_p_greater=[]
list_result_test01=[]

# ใช้ for loop สำหรับการ run ทุก replicates
for run_replicate in list_replicate:
    df0, df1, SSR0, SSR1, MSE0, MSE1, GQ_test_clusters, two_tailed, p_greater=get_result(run_replicate)

    list_df0.append(df0)
    list_df1.append(df1)
    list_SSR0.append(SSR0)
    list_SSR1.append(SSR1)
    list_MSE0.append(MSE0)
    list_MSE1.append(MSE1)
    list_GQ_test_clusters.append(GQ_test_clusters)
    list_two_tailed.append(two_tailed)
    list_p_greater.append(p_greater)

In [8]:
# สร้างตารางผลลัพธ์
alpha = 0.05
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
two_tailed_new=pd.DataFrame({"Replicate":list_replicate, "df0":list_df0, "df1":list_df1, "SSR0":list_SSR0,
                     "SSR1":list_SSR1, "MSE0":list_MSE0, "MSE1":list_MSE1, 
                     "GQ_test_clusters":list_GQ_test_clusters, "two_tailed":list_two_tailed, "greater":list_p_greater})
two_tailed_new.set_index(['Replicate'], inplace=True)
two_tailed_new=two_tailed_new[(two_tailed_new["two_tailed"]<=1) & (two_tailed_new["greater"]<=1)]
two_tailed_new["Result_test"]=two_tailed_new["two_tailed"].apply(lambda p_value: 'Reject005=0 : Heteroscedasticity' 
                                                                  if p_value < alpha else 'Reject005=1 : Homoscedasticity')
#two_tailed_new_show=two_tailed_new.drop(columns="greater")
two_tailed_new.astype(str)

,df0,df1,SSR0,SSR1,MSE0,MSE1,GQ_test_clusters,two_tailed,greater,Result_test
Replicate,,,,,,,,,,
4.0,7.0,17.0,1.1025405569411504e+19,4.1873247018971824e+20,1.8375675949019174e+18,2.617077938685739e+19,15.638359182850772,0.0011376895310313273,0.9994311552344843,Reject005=0 : Heteroscedasticity
5.0,14.0,10.0,7.003309482421591e+17,5.7477426003044506e+20,5.387161140324301e+16,6.386380667004945e+19,1149.0052896596837,2.220446049250313e-16,0.9999999999999999,Reject005=0 : Heteroscedasticity
6.0,17.0,7.0,7.764417829708533e+18,4.4382585736748296e+20,4.852761143567833e+17,7.397097622791383e+19,138.82081311231403,1.9406698470447736e-13,0.999999999999903,Reject005=0 : Heteroscedasticity
7.0,8.0,16.0,4.159069651255755e+19,1.5033368462524154e+21,5.941528073222506e+18,1.0022245641682769e+20,18.07299435101442,0.00029777877153946974,0.9998511106142303,Reject005=0 : Heteroscedasticity
9.0,13.0,11.0,5.341150159998265e+19,6.376221007351189e+19,4.4509584666652206e+18,6.376221007351188e+18,1.4108447978517655,0.5488804315141373,0.7255597842429313,Reject005=1 : Homoscedasticity
10.0,10.0,14.0,7.478758594703056e+18,3.4369248760357926e+20,8.309731771892284e+17,2.643788366181379e+19,32.825586077404864,3.6360141342228758e-06,0.9999981819929329,Reject005=0 : Heteroscedasticity
12.0,9.0,15.0,3.821551214804301e+17,2.9040227204073757e+20,4.776939018505376e+16,2.0743019431481254e+19,455.9440746193567,8.753220370749659e-11,0.9999999999562339,Reject005=0 : Heteroscedasticity
13.0,14.0,10.0,5.483443115146391e+19,2.191852941548179e+20,4.2180331654972237e+18,2.4353921572757545e+19,5.596108236613901,0.003944219314537367,0.9980278903427313,Reject005=0 : Heteroscedasticity
14.0,16.0,8.0,1.0806149607689153e+19,2.806806689014097e+20,7.204099738459436e+17,4.00972384144871e+19,51.94832185215915,1.0648577575267382e-09,0.9999999994675711,Reject005=0 : Heteroscedasticity


In [9]:
# สร้างตารางผลลัพธ์
alpha = 0.05
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
two_tailed_new=pd.DataFrame({"Replicate":list_replicate, "df0":list_df0, "df1":list_df1, "SSR0":list_SSR0,
                     "SSR1":list_SSR1, "MSE0":list_MSE0, "MSE1":list_MSE1,
                     "GQ_test_clusters":list_GQ_test_clusters, "two_tailed":list_two_tailed, "greater":list_p_greater})
two_tailed_new.set_index(['Replicate'], inplace=True)

two_tailed_new["two_tailed_new"]=two_tailed_new[["two_tailed","greater"]].apply(lambda value: value[0] if value[0]<0.5 else 2*value[1], axis=1)
two_tailed_new["Result_test"]=two_tailed_new["two_tailed_new"].apply(lambda p_value: 'Reject005=0 : Heteroscedasticity' 
                                                                      if p_value < alpha else 'Reject005=1 : Homoscedasticity')


two_tailed_new_show=two_tailed_new.drop(columns=["two_tailed", "greater"])
two_tailed_new_show.astype(str)

,df0,df1,SSR0,SSR1,MSE0,MSE1,GQ_test_clusters,two_tailed_new,Result_test
Replicate,,,,,,,,,
0.0,14.0,10.0,6.164861303045005e+20,1.8833749275525294e+20,4.7422010023423115e+19,2.0926388083916993e+19,0.42770222539656905,0.18119741096466613,Reject005=1 : Homoscedasticity
1.0,12.0,12.0,9.597182634721033e+19,2.235736616888479e+18,8.72471148611003e+18,2.0324878335349808e+17,0.02329575982851416,1.1656572971814362e-07,Reject005=0 : Heteroscedasticity
2.0,10.0,14.0,6.153601421498776e+19,9.804328178001914e+18,6.837334912776417e+18,7.541790906155318e+17,0.11380476368276018,0.00036672528857977486,Reject005=0 : Heteroscedasticity
3.0,8.0,16.0,1.966443578800908e+20,1.451092835989398e+18,2.8092051125727257e+19,9.67395223992932e+16,0.0036896376067759875,2.6826807545577515e-15,Reject005=0 : Heteroscedasticity
4.0,7.0,17.0,1.1025405569411504e+19,4.1873247018971824e+20,1.8375675949019174e+18,2.617077938685739e+19,15.638359182850772,0.0011376895310313273,Reject005=0 : Heteroscedasticity
...,...,...,...,...,...,...,...,...,...
995.0,15.0,9.0,2.2886267385519407e+20,8.586880557661905e+19,1.6347333846799577e+19,1.0733600697077381e+19,0.6253299131320261,0.48218131460245717,Reject005=1 : Homoscedasticity
996.0,8.0,16.0,1.4192551680773746e+16,7.446360710510958e+20,2027507382967678.0,4.964240473673972e+19,26233.340127971263,2.220446049250313e-16,Reject005=0 : Heteroscedasticity
997.0,11.0,13.0,9.826143188694781e+19,3.642551794234482e+17,9.826143188694782e+18,3.0354598285287348e+16,0.0031366927504701946,1.239176630814464e-13,Reject005=0 : Heteroscedasticity


In [10]:
two_tailed_new_show['Result_test'].value_counts()

Reject005=0 : Heteroscedasticity    738
Reject005=1 : Homoscedasticity      262
Name: Result_test, dtype: int64